# CNN Average Model Perfomance
This notebook was used to get four instantiations of a model with differing training and testing sampling to get an idea of how this CNN will perform on average. The performance of these four instances are used to help determine the correctness of our bias metrics. Additionally, the last set of models is used to test a potential mitigation tactic. There are five types of training and testing used to get a wider range of data biases.

The five types of training/testing splits are:
1. training sampled from 1st half of 0.1; test sampled from 2nd half of 0.1 (9900 train - 990 test)
2. training sampled from 1st half of 0.9; test sampled from 2nd half of 0.9 (9900 train - 990 test)
3. training sampled from all of 0.1; test sampled from "full/test"          (9960 train - 1000 test)
4. training sampled from all of 0.9; test sampled from "full/test"          (9960 train - 1000 test)
5. 50% training sampled from 0.1, 50% training sampled from 09; test sampled from "full/test" (9996 train - 1000 test)

Each of these types had four instantations to ensure results are not from chance of a sampling, making a total of 20 models.

Note that several changes have been made from CNN_CrossEntropy.ipynb to get this version:
- added additional convolutional layer (3 convolutional layers total)
- added adaptive average pooling layer to ensure a 5x5 shape no matter the input size and to reduce noise
- added dropout layer to randomly set 50% of features to 0 to prevent overfitting (prevent overreliance on certain neurons)
- during model training, added learning rate scheduler that will reduce the learning rate by factor of 0.5 every 10 epochs, which will speed up learning at the beginning then slow down later
- model training uses 50 epochs
- evaluate_model_classwise() function that will evaluate the classwisew accuracy of a model
- run_biased_mnist_cnn_subset() now has commented options for sampling from first/second half of a dataset for train/test
- run_biased_mnist_cnn_subset() will now return the train and test accuracies for both overall and classwise
- a small cell with a for-loop to run four model instantations
- a small cell to cleanly print out the results of the four model instantiations

The final results of the 16 models can be found at the bottom of this notebook underneath "OFFICIAL RUN FOR 4 MODELS" section.

In [288]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import json
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset, ConcatDataset
from torchvision import transforms
from tqdm import tqdm
import psutil
import threading
import time
import torchvision
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

In [228]:
"""
# Biased MNIST CNN Training - CrossEntropy Version

## Model Architecture:
- SimpleCNN with 2 convolutional layers followed by a fully connected layer
- Input: RGB images (3 channels, 160x160)
- Conv1: 3 -> 16 channels, 3x3 kernel, stride 2, padding 1, followed by ReLU and MaxPool
- Conv2: 16 -> 32 channels, 3x3 kernel, stride 2, padding 1, followed by ReLU and MaxPool
- Final feature map size: 32 x 10 x 10 
- Fully connected layer: 3200 -> 10 (one for each digit)

## Loss Function:
- CrossEntropyLoss
- Works directly with class indices (0-9)
- Good for multi-class classification problems

## Training Details:
- Optimizer: Adam with learning rate 0.001
- Batch size: 32
- Epochs: 20 for subset training (10% of training data)
- Dataset: Biased MNIST with correlation level 0.5
- Uses data normalization based on dataset statistics

## How to Run:
1. Place your biased MNIST dataset in the correct paths:
   - Default paths assume: 'biased_mnist/full_0.5/trainval'
2. Run the subset training function for faster iteration:
   - run_biased_mnist_cnn_subset()

## Expected Outputs:
- Training/validation accuracies and loss will be displayed during training
- Model will be saved as 'simple_biased_mnist_cnn_subset_05.pth'

## Notes:
- Memory monitoring is included to track RAM usage
- You can adjust the correlation level by changing paths and filenames
- The test on Standard MNIST probably isn't a good idea I am not sure yet. 
"""

"\n# Biased MNIST CNN Training - CrossEntropy Version\n\n## Model Architecture:\n- SimpleCNN with 2 convolutional layers followed by a fully connected layer\n- Input: RGB images (3 channels, 160x160)\n- Conv1: 3 -> 16 channels, 3x3 kernel, stride 2, padding 1, followed by ReLU and MaxPool\n- Conv2: 16 -> 32 channels, 3x3 kernel, stride 2, padding 1, followed by ReLU and MaxPool\n- Final feature map size: 32 x 10 x 10 \n- Fully connected layer: 3200 -> 10 (one for each digit)\n\n## Loss Function:\n- CrossEntropyLoss\n- Works directly with class indices (0-9)\n- Good for multi-class classification problems\n\n## Training Details:\n- Optimizer: Adam with learning rate 0.001\n- Batch size: 32\n- Epochs: 20 for subset training (10% of training data)\n- Dataset: Biased MNIST with correlation level 0.5\n- Uses data normalization based on dataset statistics\n\n## How to Run:\n1. Place your biased MNIST dataset in the correct paths:\n   - Default paths assume: 'biased_mnist/full_0.5/trainval'\n

In [229]:
# Memory monitoring class
class MemoryMonitor:
    def __init__(self, interval=1.0):
        self.interval = interval
        self.running = False
        self.thread = None
        self.max_memory = 0
        self.current_memory = 0
    
    def memory_monitor_func(self):
        while self.running:
            # Get memory info
            process = psutil.Process(os.getpid())
            memory_info = process.memory_info()
            memory_mb = memory_info.rss / (1024 * 1024)  # converting to mb
            
            self.current_memory = memory_mb
            self.max_memory = max(self.max_memory, memory_mb)
            time.sleep(self.interval)
    
    def start(self):
        self.running = True
        self.thread = threading.Thread(target=self.memory_monitor_func)
        self.thread.daemon = True
        self.thread.start()
    
    def stop(self):
        self.running = False
        if self.thread:
            self.thread.join(timeout=2.0)
    
    def get_memory_usage(self):
        return {
            'current': self.current_memory,
            'max': self.max_memory
        }

In [230]:
# Define the BiasedMNISTDataset class with JSON integration
class BiasedMNISTDataset(Dataset):
    def __init__(self, root_dir, transform=None, json_path=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        # load labels from JSON
        label_dict = {}
        if json_path and os.path.exists(json_path):
            try:
                with open(json_path, 'r') as f:
                    json_data = json.load(f)

                if isinstance(json_data, list):
                    for item in json_data:
                        if isinstance(item, dict) and 'index' in item and 'digit' in item:
                            label_dict[item['index']] = item['digit']
                print(f"Loaded {len(label_dict)} labels from JSON file")
            except Exception as e:
                print(f"Error loading JSON: {e}")
        
        # Load the images
        if os.path.exists(root_dir):
            for filename in os.listdir(root_dir):
                if filename.endswith('.jpg'):
                    self.image_paths.append(os.path.join(root_dir, filename))

                    try:
                        index = int(os.path.basename(filename).split('.')[0])
                        
                        # get label from JSON
                        if index in label_dict:
                            label = label_dict[index]
                        else:
                            label = index % 10
                        
                    except (ValueError, IndexError) as e:
                        print(f"Error parsing filename {filename}: {e}")
                        label = 0
                    
                    self.labels.append(label)
        else:
            print(f"Directory not found: {root_dir}")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [267]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        
        # First conv layer: 3 -> 16 channels
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        
        # Second conv layer: 16 -> 32 channels
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(32)

        # Third conv layer: 32 -> 64 channels (added)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.5)
        
        # Adaptive pooling to make input size more flexible
        self.pool = nn.AdaptiveAvgPool2d((5, 5))
        
        # Final linear layer: 64 x 5 x 5 -> num_classes
        self.fc = nn.Linear(64 * 5 * 5, num_classes)

    def forward(self, x):
        # Conv1 + BN + ReLU + MaxPool
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)

        # Conv2 + BN + ReLU + MaxPool
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        # Conv3 + BN + ReLU + MaxPool
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)

        # Adaptive pooling
        x = self.pool(x)

        # Apply dropout
        x = self.dropout(x)

        # Flatten and fully connected layer
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x


In [232]:
# Calculate dataset statistics with a small sample
def calculate_stats_fast(dataset, sample_size=1000):
    indices = torch.randperm(len(dataset))[:sample_size]
    
    # Create a temporary dataloader
    mini_loader = DataLoader(
        Subset(dataset, indices),
        batch_size=100,
        shuffle=False,
        num_workers=0
    )
    
    print(f"Calculating statistics using {sample_size} random samples (fast mode)...")
    
    channels_sum = torch.zeros(3)
    channels_squared_sum = torch.zeros(3)
    num_batches = 0
    
    # memory monitoring
    memory_monitor = MemoryMonitor()
    memory_monitor.start()
    
    try:
        # progress bar
        progress_bar = tqdm(mini_loader, desc="Calculating stats")
        
        for data, _ in progress_bar:
            # [batch_size, 3, height, width]
            channels_sum += torch.mean(data, dim=[0, 2, 3])
            channels_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
            num_batches += 1
            
            mem_usage = memory_monitor.get_memory_usage()
            progress_bar.set_postfix({'RAM': f"{mem_usage['current']:.1f}MB"})
    
    finally:
        memory_monitor.stop()

        mem_usage = memory_monitor.get_memory_usage()
        print(f"Statistics calculation - Maximum RAM usage: {mem_usage['max']:.1f}MB")
    
    mean = channels_sum / num_batches
    std = (channels_squared_sum / num_batches - mean**2)**0.5
    
    print(f"Fast statistics calculation complete. Using sample of {sample_size} images.")
    return mean, std

def create_subset_loader(dataset, fraction=0.1, batch_size=32):
    subset_size = int(len(dataset) * fraction)
    indices = torch.randperm(len(dataset))[:subset_size]
    
    subset_dataset = Subset(dataset, indices)
    
    # DataLoader
    subset_loader = DataLoader(
        subset_dataset, 
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    
    return subset_loader, subset_size

def check_label_range(dataset):
    try:
        sample_size = min(1000, len(dataset))
        indices = torch.randperm(len(dataset))[:sample_size]
        
        labels = [dataset[i.item()][1] for i in indices]
        min_label = min(labels)
        max_label = max(labels)
        
        print(f"Label range (from sample of {sample_size}): {min_label} to {max_label}")

        if max_label >= 10:
            print(f"WARNING: Found labels outside expected range (0-9): max={max_label}")
            return False
        return True
    except Exception as e:
        print(f"Error checking label range: {e}")
        return False

In [269]:
def train_model(model, train_loader, test_loader, num_epochs=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model = model.to(device)

    # Using CrossEntropyLoss
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

    # Learning rate scheduler
    scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

    # Memory monitoring setup (optional, your code seems to include this)
    memory_monitor = MemoryMonitor()
    memory_monitor.start()

    try:
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            correct = 0
            total = 0

            print(f"Epoch {epoch+1}/{num_epochs}")
            progress_bar = tqdm(train_loader, desc="Training")
            
            for inputs, labels in progress_bar:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = model(inputs)

                # Calculate CrossEntropy loss directly with integer labels
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                mem_usage = memory_monitor.get_memory_usage()

                progress_bar.set_postfix({
                    'loss': f"{loss.item():.4f}",
                    'acc': f"{100 * correct / total:.2f}%",
                    'RAM': f"{mem_usage['current']:.1f}MB"
                })

            epoch_loss = running_loss / len(train_loader)
            epoch_acc = 100 * correct / total
            mem_usage = memory_monitor.get_memory_usage()
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%, Max RAM: {mem_usage["max"]:.1f}MB')

            # Evaluate on the test set
            test_accuracy = evaluate_model(model, test_loader, device, memory_monitor)[0]
            print(f'Test Accuracy: {test_accuracy:.2f}%')

            # Step the learning rate scheduler
            scheduler.step()
    
    finally:
        memory_monitor.stop()
        mem_usage = memory_monitor.get_memory_usage()
        print(f"Maximum RAM usage: {mem_usage['max']:.1f}MB")
    
    return model


In [236]:
# version that returns class-wise accuracy as well
def evaluate_model(model, test_loader, device=None, memory_monitor=None, num_classes=10, show_progress=True):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.eval()
    total_correct = 0
    total_samples = 0

    class_correct = [0 for _ in range(num_classes)]
    class_total = [0 for _ in range(num_classes)]

    progress_iter = test_loader
    if show_progress:
        progress_iter = tqdm(test_loader, desc="Evaluating")

    with torch.no_grad():
        for inputs, labels in progress_iter:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

            for label, prediction in zip(labels, predicted):
                class_total[label.item()] += 1
                if label == prediction:
                    class_correct[label.item()] += 1

            if show_progress:
                postfix = {'acc': f"{100 * total_correct / total_samples:.2f}%"}
                if memory_monitor:
                    mem_usage = memory_monitor.get_memory_usage()
                    postfix['RAM'] = f"{mem_usage['current']:.1f}MB"
                progress_iter.set_postfix(postfix)

    accuracy = 100 * total_correct / total_samples
    classwise_accuracy = {
        f"class_{i}": 100 * class_correct[i] / class_total[i] if class_total[i] > 0 else 0.0
        for i in range(num_classes)
    }

    return accuracy, classwise_accuracy

In [237]:
# evalute model on train and test set; obtain overall accuracies and class-wise accuracies
def evaluate_model_classwise(model, train_loader, test_loader):
    train_accuracy, train_classwise = evaluate_model(model, train_loader, device=None, memory_monitor=None, num_classes=10)
    test_accuracy, test_classwise = evaluate_model(model, test_loader, device=None, memory_monitor=None, num_classes=10)

    print(f"Train Accuracy: {train_accuracy:.2f}%")
    print("Train Class-wise Accuracy:")
    for cls, acc in train_classwise.items():
        print(f"  {cls}: {acc:.2f}%")

    print(f"Test Accuracy: {test_accuracy:.2f}%")
    print("Test Class-wise Accuracy:")
    for cls, acc in test_classwise.items():
        print(f"  {cls}: {acc:.2f}%")
    
    return train_accuracy, train_classwise, test_accuracy, test_classwise

In [238]:
# CHANGES MADE:
# - Removed "mse" from the model filename

# Run the complete workflow with full dataset
def run_biased_mnist_cnn():
    # CHANGE THIS PATH AND RENAME MODEL FILE FOR EACH BIAS LEVEL
    base_dir = 'biased_mnist'
    train_folder = f"{base_dir}/full_0.5/trainval"  # Using full_0.5 for now
    test_folder = f"{base_dir}/full/test"
    train_json_path = f"{base_dir}/full_0.5/trainval.json" # Using full_0.5 for now
    test_json_path = f"{base_dir}/full/test.json"
    
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    
    print("Creating datasets...")
    train_dataset = BiasedMNISTDataset(train_folder, transform=transform, json_path=train_json_path)
    test_dataset = BiasedMNISTDataset(test_folder, transform=transform, json_path=test_json_path)

    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")
    
    if len(train_dataset) == 0 or len(test_dataset) == 0:
        print("Error: Empty dataset found. Please check the file paths.")
        return

    print("Checking label ranges...")
    train_labels_ok = check_label_range(train_dataset)
    test_labels_ok = check_label_range(test_dataset)
    
    if not (train_labels_ok and test_labels_ok):
        print("WARNING: Label range check failed. Please check the dataset.")

    mean, std = calculate_stats_fast(train_dataset, sample_size=1000)
    print(f"Dataset mean: {mean}")
    print(f"Dataset std: {std}")
    
    normalized_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])
    
    train_dataset_normalized = BiasedMNISTDataset(train_folder, transform=normalized_transform, json_path=train_json_path)
    test_dataset_normalized = BiasedMNISTDataset(test_folder, transform=normalized_transform, json_path=test_json_path)
    
    # dataloaders
    train_loader = DataLoader(train_dataset_normalized, batch_size=32, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_dataset_normalized, batch_size=32, shuffle=False, num_workers=0)
    
    print(f"Using FULL training dataset with {len(train_dataset_normalized)} images")
    print(f"Using FULL test dataset with {len(test_dataset_normalized)} images")
    
    # the simplified CNN model
    model = SimpleCNN(num_classes=10)
    print("Model architecture:")
    print(model)
    
    # Train the model
    print("\nStarting training...")
    trained_model = train_model(model, train_loader, test_loader, num_epochs=3)
    
    # Final EVAL
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    final_accuracy, classwise_acc = evaluate_model(trained_model, test_loader, device, None, num_classes=10)
    print(f"\nFinal Test Accuracy: {final_accuracy:.2f}%")
    print(f"\nFinal Classwise Accuracy: {classwise_acc:.2f}")
    
    # Save model
    torch.save(trained_model.state_dict(), 'simple_biased_mnist_cnn_full.pth')
    print("Model saved as 'simple_biased_mnist_cnn_full.pth'")

In [293]:
# CHANGES MADE:
# - Removed "mse" from the model filename

# Run the complete workflow with subset of data
def run_biased_mnist_cnn_subset():
    # SET PATHS CORRECTLY FOR CORRECT BIAS LEVEL
    base_dir = 'biased_mnist'
    #train_folder = f"{base_dir}/full_0.9/trainval"  # Using full_0.5 for now
    #test_folder = f"{base_dir}/full/test"
    #test_folder = f"{base_dir}/full_0.9/trainval"
    #train_json_path = f"{base_dir}/full_0.9/trainval.json" # Change when changing correlation levels
    #test_json_path = f"{base_dir}/full/test.json"
    #test_json_path = f"{base_dir}/full_0.9/trainval.json"
    train_folder1 = f"{base_dir}/full_0.1/trainval"
    train_folder2 = f"{base_dir}/full_0.9/trainval"
    test_folder = f"{base_dir}/full/test"
    train_json_path1 = f"{base_dir}/full_0.1/trainval.json"
    train_json_path2 = f"{base_dir}/full_0.9/trainval.json"
    test_json_path = f"{base_dir}/full/test.json"
    
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    
    print("Creating datasets...")
    train_dataset1 = BiasedMNISTDataset(train_folder1, transform=transform, json_path=train_json_path1)
    train_dataset2 = BiasedMNISTDataset(train_folder2, transform=transform, json_path=train_json_path2)
    test_dataset = BiasedMNISTDataset(test_folder, transform=transform, json_path=test_json_path)
    train_dataset = ConcatDataset([train_dataset1, train_dataset2]) 


    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")
    
    if len(train_dataset) == 0 or len(test_dataset) == 0:
        print("Error: Empty dataset found. Please check the file paths.")
        return

    print("Checking label ranges...")
    train_labels_ok = check_label_range(train_dataset)
    test_labels_ok = check_label_range(test_dataset)
    
    if not (train_labels_ok and test_labels_ok):
        print("WARNING: Label range check failed. Please check the dataset.")

    mean, std = calculate_stats_fast(train_dataset, sample_size=1000)
    print(f"Dataset mean: {mean}")
    print(f"Dataset std: {std}")

    normalized_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])
    
    train_dataset_normalized1 = BiasedMNISTDataset(train_folder1, transform=normalized_transform, json_path=train_json_path1)
    train_dataset_normalized2 = BiasedMNISTDataset(train_folder2, transform=normalized_transform, json_path=train_json_path2)
    train_dataset_normalized = ConcatDataset([train_dataset_normalized1, train_dataset_normalized2]) 
    test_dataset_normalized = BiasedMNISTDataset(test_folder, transform=normalized_transform, json_path=test_json_path)

    train_fraction = 0.0833  # Use 10% of training data (10,000 IMAGES)
    #test_fraction = 0.2   # Use 20% of test data (2,000 IMAGES)
    test_fraction = 0.1   # Use 10% of test data (1,000 IMAGES)
    #train_fraction = 0.33   # use 9900 of train data from 0.1 set
    #test_fraction = 0.033   # Use 1% of test data (9900 IMAGES) from same 0.1 set

    # Split indices
    half_len_train = len(train_dataset_normalized) // 2
    half_len_test = len(test_dataset_normalized) // 2

    # Create Subsets
    #train_half_dataset = Subset(train_dataset_normalized, list(range(half_len_train)))
    #test_half_dataset = Subset(test_dataset_normalized, list(range(half_len_test, len(test_dataset_normalized))))
    
    train_loader, train_subset_size = create_subset_loader(
        train_dataset_normalized, 
        #train_half_dataset, # only sample from first half of dataset for train
        fraction=train_fraction, 
        batch_size=32
    )
    
    test_loader, test_subset_size = create_subset_loader(
        test_dataset_normalized, 
        #test_half_dataset, #only sample from second half
        fraction=test_fraction, 
        batch_size=32
    )
    
    print(f"Using {train_subset_size} training images ({train_fraction*100:.1f}% of dataset)")
    print(f"Using {test_subset_size} test images ({test_fraction*100:.1f}% of dataset)")
    
    # the simplified CNN model
    model = SimpleCNN(num_classes=10)
    print("Model architecture:")
    print(model)
    
    # Train the model
    print("\nStarting training...")
    trained_model = train_model(model, train_loader, test_loader, num_epochs=50)
    
    # Final EVAL
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    final_accuracy, classwise_acc = evaluate_model(trained_model, test_loader, device, None, num_classes=10)
    print(f"\nFinal Test Accuracy: {final_accuracy:.2f}%")
    print(f"\nFinal Classwise Accuracy: {classwise_acc}")
    
    # Save model
    torch.save(trained_model.state_dict(), 'simple_biased_mnist_cnn_subset_01.pth')
    print("Model saved as 'simple_biased_mnist_cnn_subset_01.pth'")

    # return accuracies
    return evaluate_model_classwise(trained_model, train_loader, test_loader)

In [240]:
def load_trained_model(model_path):
    model = SimpleCNN(num_classes=10)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

def load_standard_mnist():
    # Standard normalization for MNIST
    transform = transforms.Compose([
        transforms.Resize((160, 160)),  # Resize to match our biased MNIST images
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    mnist_test = torchvision.datasets.MNIST(
        root='./data', 
        train=False, 
        download=True, 
        transform=transform
    )
    
    class MNISTtoRGB(torch.utils.data.Dataset):
        def __init__(self, mnist_dataset):
            self.mnist_dataset = mnist_dataset
            
        def __len__(self):
            return len(self.mnist_dataset)
            
        def __getitem__(self, idx):
            img, label = self.mnist_dataset[idx]
            rgb_img = torch.cat([img, img, img], dim=0)
            return rgb_img, label
    
    rgb_mnist_test = MNISTtoRGB(mnist_test)
    return rgb_mnist_test

In [241]:
# CHANGES MADE:
# - Removed "mse" from model filenames

def test_on_standard_mnist():
    model_01_path = 'simple_biased_mnist_cnn_subset_01.pth'
    model_05_path = 'simple_biased_mnist_cnn_subset_05.pth'
    try:
        model_01 = load_trained_model(model_01_path)
        print(f"Successfully loaded model trained on correlation level 0.1")
    except:
        print(f"Could not load model from {model_01_path}")
        model_01 = None
        
    try:
        model_05 = load_trained_model(model_05_path)
        print(f"Successfully loaded model trained on correlation level 0.5")
    except:
        print(f"Could not load model from {model_05_path}")
        model_05 = None

    print("Loading standard MNIST test set...")
    mnist_dataset = load_standard_mnist()
    mnist_loader = DataLoader(mnist_dataset, batch_size=64, shuffle=False, num_workers=0)
    print(f"Loaded {len(mnist_dataset)} standard MNIST test images")

    print("\nEvaluating models on standard (unbiased) MNIST:")
    
    results = {}
    
    if model_01 is not None:
        print("\nEvaluating model trained on correlation level 0.1:")
        accuracy_01 = evaluate_model(model_01, mnist_loader, None, None, num_classes=10)[0]
        results["0.1"] = accuracy_01
        print(f"Accuracy on standard MNIST: {accuracy_01:.2f}%")
        
    if model_05 is not None:
        print("\nEvaluating model trained on correlation level 0.5:")
        accuracy_05 = evaluate_model(model_05, mnist_loader, None, None, num_classes=10)[0]
        results["0.5"] = accuracy_05
        print(f"Accuracy on standard MNIST: {accuracy_05:.2f}%")
    
    # Print summary
    print("\nSummary of model performance on standard MNIST:")
    print("-" * 50)
    print("| Correlation Level | Biased Test Acc | Standard MNIST Acc |")
    print("|-------------------|-----------------|-------------------|")
    
    if "0.1" in results:
        print(f"| 0.1               | 37.55%          | {results['0.1']:.2f}%             |")
    
    if "0.5" in results:
        print(f"| 0.5               | 18.00%          | {results['0.5']:.2f}%             |")
    
    print("-" * 50)
    print("Note: A model that relies heavily on bias features will perform poorly on standard MNIST")

In [242]:
# Execute the function for full dataset training
# run_biased_mnist_cnn()

# Execute the function for subset training
#run_biased_mnist_cnn_subset()

# Run the test on standard mnist as counterfactual set
# test_on_standard_mnist()

In [294]:
tot_train_accuracies = []
classwise_train_accuracies = []
tot_test_accuracies = []
classwise_test_accuracies = []
num_models = 4

for i in range(num_models):
    trainA, class_trainA, testA, class_testA = run_biased_mnist_cnn_subset()
    tot_train_accuracies.append(trainA)
    classwise_train_accuracies.append(list(class_trainA.values()))
    tot_test_accuracies.append(testA)
    classwise_test_accuracies.append(list(class_testA.values()))

Creating datasets...
Loaded 60000 labels from JSON file
Loaded 60000 labels from JSON file
Loaded 10000 labels from JSON file
Training dataset size: 120000
Test dataset size: 10000
Checking label ranges...
Label range (from sample of 1000): 0 to 9
Label range (from sample of 1000): 0 to 9
Calculating statistics using 1000 random samples (fast mode)...


Calculating stats: 100%|███████████| 10/10 [00:00<00:00, 12.01it/s, RAM=471.6MB]


Statistics calculation - Maximum RAM usage: 471.6MB
Fast statistics calculation complete. Using sample of 1000 images.
Dataset mean: tensor([0.0601, 0.0498, 0.0577])
Dataset std: tensor([0.1498, 0.1411, 0.1414])
Loaded 60000 labels from JSON file
Loaded 60000 labels from JSON file
Loaded 10000 labels from JSON file
Using 9996 training images (8.3% of dataset)
Using 1000 test images (10.0% of dataset)
Model architecture:
SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (pool): AdaptiveAvgPool

Training: 100%|█| 313/313 [00:26<00:00, 11.76it/s, loss=1.8317, acc=46.43%, RAM=


Epoch 1/50, Loss: 1.7501, Accuracy: 46.43%, Max RAM: 551.9MB


...

Test Accuracy: 61.80%
Maximum RAM usage: 533.4MB


Evaluating: 100%|███████████████████| 32/32 [00:01<00:00, 16.41it/s, acc=61.80%]



Final Test Accuracy: 61.80%

Final Classwise Accuracy: {'class_0': 65.59139784946237, 'class_1': 66.93548387096774, 'class_2': 62.83185840707964, 'class_3': 69.72477064220183, 'class_4': 67.74193548387096, 'class_5': 56.043956043956044, 'class_6': 61.61616161616162, 'class_7': 61.05263157894737, 'class_8': 53.65853658536585, 'class_9': 49.504950495049506}
Model saved as 'simple_biased_mnist_cnn_subset_01.pth'


Evaluating: 100%|███████████████████| 32/32 [00:03<00:00, 10.48it/s, acc=61.80%]


Train Accuracy: 99.93%
Train Class-wise Accuracy:
  class_0: 100.00%
  class_1: 100.00%
  class_2: 99.90%
  class_3: 99.91%
  class_4: 99.80%
  class_5: 100.00%
  class_6: 100.00%
  class_7: 100.00%
  class_8: 99.68%
  class_9: 100.00%
Test Accuracy: 61.80%
Test Class-wise Accuracy:
  class_0: 65.59%
  class_1: 66.94%
  class_2: 62.83%
  class_3: 69.72%
  class_4: 67.74%
  class_5: 56.04%
  class_6: 61.62%
  class_7: 61.05%
  class_8: 53.66%
  class_9: 49.50%


In [295]:
for i in range(num_models):
    print("*********************************")
    print(f"   ACCURACIES FOR MODEL {i+1}   ")
    print("Overall Train\t| Overall Test")
    print(f"{tot_train_accuracies[i]:.2f}%\t\t| {tot_test_accuracies[i]:.2f}%")
    print("----------------------------")
    print("Train Classwise\t| Test Classwise")
    for j in range(10):
        print(f"{j}: {classwise_train_accuracies[i][j]:.2f}%\t| {j}: {classwise_test_accuracies[i][j]:.2f}%")
    print("*********************************")

*********************************
   ACCURACIES FOR MODEL 1   
Overall Train	| Overall Test
99.92%		| 61.80%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 65.18%
1: 100.00%	| 1: 75.61%
2: 100.00%	| 2: 51.40%
3: 99.70%	| 3: 63.64%
4: 99.90%	| 4: 65.56%
5: 99.90%	| 5: 56.79%
6: 99.90%	| 6: 73.47%
7: 100.00%	| 7: 56.86%
8: 99.78%	| 8: 53.85%
9: 100.00%	| 9: 51.55%
*********************************
*********************************
   ACCURACIES FOR MODEL 2   
Overall Train	| Overall Test
99.95%		| 59.30%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 55.10%
1: 100.00%	| 1: 76.86%
2: 99.90%	| 2: 51.38%
3: 99.90%	| 3: 69.52%
4: 100.00%	| 4: 57.45%
5: 100.00%	| 5: 45.33%
6: 99.90%	| 6: 71.91%
7: 100.00%	| 7: 55.93%
8: 99.79%	| 8: 46.81%
9: 100.00%	| 9: 56.70%
*********************************
*********************************
   ACCURACIES FOR MODEL 3   
Overall Train	| Overall Test
99.97%		| 58.90%
----------------------------
T

In [297]:
print(tot_train_accuracies)
print(classwise_train_accuracies)
print(tot_test_accuracies)
print(classwise_test_accuracies)

[99.91996798719488, 99.9499799919968, 99.96998799519808, 99.92997198879551]
[[100.0, 100.0, 100.0, 99.7005988023952, 99.90118577075098, 99.89669421487604, 99.89711934156378, 100.0, 99.78354978354979, 100.0], [100.0, 100.0, 99.90049751243781, 99.90319457889642, 100.0, 100.0, 99.89722507708119, 100.0, 99.79188345473464, 100.0], [100.0, 100.0, 99.89888776541962, 100.0, 100.0, 100.0, 100.0, 100.0, 99.90176817288801, 99.8984771573604], [100.0, 100.0, 99.90375360923966, 99.90503323836657, 99.79550102249489, 100.0, 100.0, 100.0, 99.68152866242038, 100.0]]
[61.8, 59.3, 58.9, 61.8]
[[65.17857142857143, 75.60975609756098, 51.401869158878505, 63.63636363636363, 65.55555555555556, 56.79012345679013, 73.46938775510205, 56.86274509803921, 53.84615384615385, 51.54639175257732], [55.10204081632653, 76.85950413223141, 51.37614678899082, 69.52380952380952, 57.4468085106383, 45.333333333333336, 71.91011235955057, 55.932203389830505, 46.808510638297875, 56.70103092783505], [59.61538461538461, 78.761061946

 # OFFICIAL RUNS FOR 4 MODELS

using 50/50 split of 0.1, newest arch, dropout rate = 0.5, scheduler learning rate step size = 10
Using 9900 training images (33.0% of dataset)
Using 990 test images (3.3% of dataset)
[99.62626262626263, 99.51515151515152, 99.61616161616162, 99.74747474747475]
[[99.89858012170386, 99.9107939339875, 99.69040247678019, 99.51028403525955, 99.46004319654428, 99.41176470588235, 99.60474308300395, 99.90300678952474, 99.39577039274924, 99.39455095862765], [99.69356486210418, 99.72652689152234, 99.6023856858847, 99.71098265895954, 99.07502569373072, 99.66517857142857, 100.0, 99.6003996003996, 98.82854100106496, 99.19354838709677], [99.46294307196563, 99.91007194244604, 99.60591133004927, 99.8995983935743, 99.69167523124358, 99.32885906040268, 99.79338842975207, 99.7104247104247, 99.29506545820745, 99.38900203665987], [99.59839357429719, 100.0, 99.89583333333333, 99.8015873015873, 99.68085106382979, 99.57446808510639, 99.70588235294117, 100.0, 99.57671957671958, 99.59390862944163]]
[69.8989898989899, 68.68686868686869, 71.61616161616162, 72.62626262626263]
[[80.68181818181819, 83.6734693877551, 53.93258426966292, 70.43478260869566, 67.88990825688073, 67.05882352941177, 83.17757009345794, 70.10309278350516, 56.73076923076923, 64.28571428571429], [71.95121951219512, 71.875, 71.42857142857143, 67.5925925925926, 73.33333333333333, 62.365591397849464, 78.57142857142857, 72.64150943396227, 46.236559139784944, 68.90756302521008], [71.875, 93.57798165137615, 62.857142857142854, 67.88990825688073, 80.0, 62.244897959183675, 77.41935483870968, 71.0, 57.77777777777778, 69.47368421052632], [79.34782608695652, 79.33884297520662, 64.51612903225806, 70.96774193548387, 71.27659574468085, 69.62025316455696, 83.49514563106796, 75.96153846153847, 60.0, 68.96551724137932]]

*********************************
   ACCURACIES FOR MODEL 1   
Overall Train	| Overall Test
99.63%		| 69.90%
----------------------------
Train Classwise	| Test Classwise
0: 99.90%	| 0: 80.68%
1: 99.91%	| 1: 83.67%
2: 99.69%	| 2: 53.93%
3: 99.51%	| 3: 70.43%
4: 99.46%	| 4: 67.89%
5: 99.41%	| 5: 67.06%
6: 99.60%	| 6: 83.18%
7: 99.90%	| 7: 70.10%
8: 99.40%	| 8: 56.73%
9: 99.39%	| 9: 64.29%
*********************************
*********************************
   ACCURACIES FOR MODEL 2   
Overall Train	| Overall Test
99.52%		| 68.69%
----------------------------
Train Classwise	| Test Classwise
0: 99.69%	| 0: 71.95%
1: 99.73%	| 1: 71.88%
2: 99.60%	| 2: 71.43%
3: 99.71%	| 3: 67.59%
4: 99.08%	| 4: 73.33%
5: 99.67%	| 5: 62.37%
6: 100.00%	| 6: 78.57%
7: 99.60%	| 7: 72.64%
8: 98.83%	| 8: 46.24%
9: 99.19%	| 9: 68.91%
*********************************
*********************************
   ACCURACIES FOR MODEL 3   
Overall Train	| Overall Test
99.62%		| 71.62%
----------------------------
Train Classwise	| Test Classwise
0: 99.46%	| 0: 71.88%
1: 99.91%	| 1: 93.58%
2: 99.61%	| 2: 62.86%
3: 99.90%	| 3: 67.89%
4: 99.69%	| 4: 80.00%
5: 99.33%	| 5: 62.24%
6: 99.79%	| 6: 77.42%
7: 99.71%	| 7: 71.00%
8: 99.30%	| 8: 57.78%
9: 99.39%	| 9: 69.47%
*********************************
*********************************
   ACCURACIES FOR MODEL 4   
Overall Train	| Overall Test
99.75%		| 72.63%
----------------------------
Train Classwise	| Test Classwise
0: 99.60%	| 0: 79.35%
1: 100.00%	| 1: 79.34%
2: 99.90%	| 2: 64.52%
3: 99.80%	| 3: 70.97%
4: 99.68%	| 4: 71.28%
5: 99.57%	| 5: 69.62%
6: 99.71%	| 6: 83.50%
7: 100.00%	| 7: 75.96%
8: 99.58%	| 8: 60.00%
9: 99.59%	| 9: 68.97%
*********************************

using 50/50 split of 0.9, newest arch, dropout rate = 0.5, scheduler learning rate step size = 10
Using 9900 training images (33.0% of dataset)
Using 990 test images (3.3% of dataset)
[100.0, 100.0, 100.0, 99.98989898989899]
[[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0], [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0], [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0], [100.0, 100.0, 100.0, 99.89949748743719, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]]
[97.77777777777777, 97.97979797979798, 97.27272727272727, 97.27272727272727]
[[94.56521739130434, 94.69026548672566, 97.97979797979798, 98.14814814814815, 100.0, 95.45454545454545, 100.0, 100.0, 97.89473684210526, 99.00990099009901], [97.24770642201835, 97.36842105263158, 96.84210526315789, 94.56521739130434, 97.9381443298969, 97.9381443298969, 97.67441860465117, 100.0, 100.0, 100.0], [96.42857142857143, 96.36363636363636, 96.875, 97.9381443298969, 99.02912621359224, 96.66666666666667, 96.15384615384616, 97.97979797979798, 97.67441860465117, 97.84946236559139], [92.78350515463917, 97.47899159663865, 98.0, 95.0, 97.84946236559139, 97.19626168224299, 97.16981132075472, 96.62921348314607, 100.0, 100.0]]
*********************************
   ACCURACIES FOR MODEL 1   
Overall Train	| Overall Test
100.00%		| 97.78%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 94.57%
1: 100.00%	| 1: 94.69%
2: 100.00%	| 2: 97.98%
3: 100.00%	| 3: 98.15%
4: 100.00%	| 4: 100.00%
5: 100.00%	| 5: 95.45%
6: 100.00%	| 6: 100.00%
7: 100.00%	| 7: 100.00%
8: 100.00%	| 8: 97.89%
9: 100.00%	| 9: 99.01%
*********************************
*********************************
   ACCURACIES FOR MODEL 2   
Overall Train	| Overall Test
100.00%		| 97.98%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 97.25%
1: 100.00%	| 1: 97.37%
2: 100.00%	| 2: 96.84%
3: 100.00%	| 3: 94.57%
4: 100.00%	| 4: 97.94%
5: 100.00%	| 5: 97.94%
6: 100.00%	| 6: 97.67%
7: 100.00%	| 7: 100.00%
8: 100.00%	| 8: 100.00%
9: 100.00%	| 9: 100.00%
*********************************
*********************************
   ACCURACIES FOR MODEL 3   
Overall Train	| Overall Test
100.00%		| 97.27%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 96.43%
1: 100.00%	| 1: 96.36%
2: 100.00%	| 2: 96.88%
3: 100.00%	| 3: 97.94%
4: 100.00%	| 4: 99.03%
5: 100.00%	| 5: 96.67%
6: 100.00%	| 6: 96.15%
7: 100.00%	| 7: 97.98%
8: 100.00%	| 8: 97.67%
9: 100.00%	| 9: 97.85%
*********************************
*********************************
   ACCURACIES FOR MODEL 4   
Overall Train	| Overall Test
99.99%		| 97.27%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 92.78%
1: 100.00%	| 1: 97.48%
2: 100.00%	| 2: 98.00%
3: 99.90%	| 3: 95.00%
4: 100.00%	| 4: 97.85%
5: 100.00%	| 5: 97.20%
6: 100.00%	| 6: 97.17%
7: 100.00%	| 7: 96.63%
8: 100.00%	| 8: 100.00%
9: 100.00%	| 9: 100.00%
*********************************

using 0.1 and full/test set, newest arch, dropout rate = 0.5, scheduler learning rate step size = 10
Using 9960 training images (16.6% of dataset)
Using 1000 test images (10.0% of dataset)
[99.66867469879519, 99.41767068273093, 99.66867469879519, 99.60843373493977]
[[99.59432048681542, 99.91063449508489, 100.0, 99.39148073022312, 99.36908517350157, 99.22737306843267, 99.89722507708119, 99.80824544582934, 99.79959919839679, 99.59308240081384], [99.09456740442656, 99.53789279112755, 99.49647532729104, 99.5136186770428, 99.38461538461539, 99.00881057268722, 99.89868287740629, 99.60591133004927, 98.96157840083073, 99.60591133004927], [99.70414201183432, 100.0, 99.9001996007984, 99.39271255060729, 99.5987963891675, 99.35553168635876, 100.0, 99.80119284294234, 99.2964824120603, 99.58246346555323], [99.79166666666667, 99.91197183098592, 99.70588235294117, 99.52651515151516, 99.56756756756756, 99.42196531791907, 99.79879275653923, 99.80601357904946, 99.19273461150352, 99.28716904276986]]
[75.0, 72.6, 75.5, 75.2]
[[72.52747252747253, 90.08264462809917, 77.45098039215686, 65.26315789473684, 72.63157894736842, 69.04761904761905, 81.11111111111111, 77.11864406779661, 69.0721649484536, 71.02803738317758], [72.34042553191489, 83.62068965517241, 72.22222222222223, 78.84615384615384, 71.56862745098039, 59.57446808510638, 82.55813953488372, 73.7864077669903, 62.37623762376238, 67.3913043478261], [71.76470588235294, 88.49557522123894, 68.26923076923077, 81.13207547169812, 82.4074074074074, 77.21518987341773, 81.31868131868131, 73.07692307692308, 64.81481481481481, 65.68627450980392], [83.50515463917526, 86.06557377049181, 79.0, 75.22123893805309, 76.59574468085107, 69.0909090909091, 81.52173913043478, 77.10843373493977, 56.12244897959184, 65.93406593406593]]
*********************************
   ACCURACIES FOR MODEL 1   
Overall Train	| Overall Test
99.67%		| 75.00%
----------------------------
Train Classwise	| Test Classwise
0: 99.59%	| 0: 72.53%
1: 99.91%	| 1: 90.08%
2: 100.00%	| 2: 77.45%
3: 99.39%	| 3: 65.26%
4: 99.37%	| 4: 72.63%
5: 99.23%	| 5: 69.05%
6: 99.90%	| 6: 81.11%
7: 99.81%	| 7: 77.12%
8: 99.80%	| 8: 69.07%
9: 99.59%	| 9: 71.03%
*********************************
*********************************
   ACCURACIES FOR MODEL 2   
Overall Train	| Overall Test
99.42%		| 72.60%
----------------------------
Train Classwise	| Test Classwise
0: 99.09%	| 0: 72.34%
1: 99.54%	| 1: 83.62%
2: 99.50%	| 2: 72.22%
3: 99.51%	| 3: 78.85%
4: 99.38%	| 4: 71.57%
5: 99.01%	| 5: 59.57%
6: 99.90%	| 6: 82.56%
7: 99.61%	| 7: 73.79%
8: 98.96%	| 8: 62.38%
9: 99.61%	| 9: 67.39%
*********************************
*********************************
   ACCURACIES FOR MODEL 3   
Overall Train	| Overall Test
99.67%		| 75.50%
----------------------------
Train Classwise	| Test Classwise
0: 99.70%	| 0: 71.76%
1: 100.00%	| 1: 88.50%
2: 99.90%	| 2: 68.27%
3: 99.39%	| 3: 81.13%
4: 99.60%	| 4: 82.41%
5: 99.36%	| 5: 77.22%
6: 100.00%	| 6: 81.32%
7: 99.80%	| 7: 73.08%
8: 99.30%	| 8: 64.81%
9: 99.58%	| 9: 65.69%
*********************************
*********************************
   ACCURACIES FOR MODEL 4   
Overall Train	| Overall Test
99.61%		| 75.20%
----------------------------
Train Classwise	| Test Classwise
0: 99.79%	| 0: 83.51%
1: 99.91%	| 1: 86.07%
2: 99.71%	| 2: 79.00%
3: 99.53%	| 3: 75.22%
4: 99.57%	| 4: 76.60%
5: 99.42%	| 5: 69.09%
6: 99.80%	| 6: 81.52%
7: 99.81%	| 7: 77.11%
8: 99.19%	| 8: 56.12%
9: 99.29%	| 9: 65.93%
*********************************

using 0.9 and full/test set, newest arch, dropout rate = 0.5, scheduler learning rate step size = 10
Using 9960 training images (16.6% of dataset)
Using 1000 test images (10.0% of dataset)
[100.0, 100.0, 100.0, 100.0]
[[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0], [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0], [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0], [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]]
[16.5, 15.2, 15.7, 16.4]
[[19.54022988505747, 15.0, 12.037037037037036, 16.037735849056602, 19.51219512195122, 14.606741573033707, 20.952380952380953, 18.103448275862068, 15.74074074074074, 13.924050632911392], [13.25301204819277, 14.40677966101695, 6.896551724137931, 13.043478260869565, 16.50485436893204, 9.090909090909092, 24.175824175824175, 26.605504587155963, 17.346938775510203, 10.784313725490197], [11.578947368421053, 13.675213675213675, 15.88785046728972, 11.627906976744185, 11.403508771929825, 18.39080459770115, 26.881720430107528, 18.181818181818183, 19.753086419753085, 11.818181818181818], [24.46808510638298, 10.56910569105691, 12.121212121212121, 16.853932584269664, 15.238095238095237, 12.631578947368421, 14.953271028037383, 24.50980392156863, 16.49484536082474, 17.97752808988764]]
*********************************
   ACCURACIES FOR MODEL 1   
Overall Train	| Overall Test
100.00%		| 16.50%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 19.54%
1: 100.00%	| 1: 15.00%
2: 100.00%	| 2: 12.04%
3: 100.00%	| 3: 16.04%
4: 100.00%	| 4: 19.51%
5: 100.00%	| 5: 14.61%
6: 100.00%	| 6: 20.95%
7: 100.00%	| 7: 18.10%
8: 100.00%	| 8: 15.74%
9: 100.00%	| 9: 13.92%
*********************************
*********************************
   ACCURACIES FOR MODEL 2   
Overall Train	| Overall Test
100.00%		| 15.20%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 13.25%
1: 100.00%	| 1: 14.41%
2: 100.00%	| 2: 6.90%
3: 100.00%	| 3: 13.04%
4: 100.00%	| 4: 16.50%
5: 100.00%	| 5: 9.09%
6: 100.00%	| 6: 24.18%
7: 100.00%	| 7: 26.61%
8: 100.00%	| 8: 17.35%
9: 100.00%	| 9: 10.78%
*********************************
*********************************
   ACCURACIES FOR MODEL 3   
Overall Train	| Overall Test
100.00%		| 15.70%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 11.58%
1: 100.00%	| 1: 13.68%
2: 100.00%	| 2: 15.89%
3: 100.00%	| 3: 11.63%
4: 100.00%	| 4: 11.40%
5: 100.00%	| 5: 18.39%
6: 100.00%	| 6: 26.88%
7: 100.00%	| 7: 18.18%
8: 100.00%	| 8: 19.75%
9: 100.00%	| 9: 11.82%
*********************************
*********************************
   ACCURACIES FOR MODEL 4   
Overall Train	| Overall Test
100.00%		| 16.40%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 24.47%
1: 100.00%	| 1: 10.57%
2: 100.00%	| 2: 12.12%
3: 100.00%	| 3: 16.85%
4: 100.00%	| 4: 15.24%
5: 100.00%	| 5: 12.63%
6: 100.00%	| 6: 14.95%
7: 100.00%	| 7: 24.51%
8: 100.00%	| 8: 16.49%
9: 100.00%	| 9: 17.98%
*********************************

50% train from 0.1, 50% train from 0.9, test from full/test
Using 9996 training images (8.3% of dataset)
Using 1000 test images (10.0% of dataset)
[99.91996798719488, 99.9499799919968, 99.96998799519808, 99.92997198879551]
[[100.0, 100.0, 100.0, 99.7005988023952, 99.90118577075098, 99.89669421487604, 99.89711934156378, 100.0, 99.78354978354979, 100.0], [100.0, 100.0, 99.90049751243781, 99.90319457889642, 100.0, 100.0, 99.89722507708119, 100.0, 99.79188345473464, 100.0], [100.0, 100.0, 99.89888776541962, 100.0, 100.0, 100.0, 100.0, 100.0, 99.90176817288801, 99.8984771573604], [100.0, 100.0, 99.90375360923966, 99.90503323836657, 99.79550102249489, 100.0, 100.0, 100.0, 99.68152866242038, 100.0]]
[61.8, 59.3, 58.9, 61.8]
[[65.17857142857143, 75.60975609756098, 51.401869158878505, 63.63636363636363, 65.55555555555556, 56.79012345679013, 73.46938775510205, 56.86274509803921, 53.84615384615385, 51.54639175257732], [55.10204081632653, 76.85950413223141, 51.37614678899082, 69.52380952380952, 57.4468085106383, 45.333333333333336, 71.91011235955057, 55.932203389830505, 46.808510638297875, 56.70103092783505], [59.61538461538461, 78.76106194690266, 58.333333333333336, 56.32183908045977, 56.52173913043478, 52.04081632653061, 67.46987951807229, 58.11965811965812, 52.04081632653061, 48.0], [65.59139784946237, 66.93548387096774, 62.83185840707964, 69.72477064220183, 67.74193548387096, 56.043956043956044, 61.61616161616162, 61.05263157894737, 53.65853658536585, 49.504950495049506]]
*********************************
   ACCURACIES FOR MODEL 1   
Overall Train	| Overall Test
99.92%		| 61.80%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 65.18%
1: 100.00%	| 1: 75.61%
2: 100.00%	| 2: 51.40%
3: 99.70%	| 3: 63.64%
4: 99.90%	| 4: 65.56%
5: 99.90%	| 5: 56.79%
6: 99.90%	| 6: 73.47%
7: 100.00%	| 7: 56.86%
8: 99.78%	| 8: 53.85%
9: 100.00%	| 9: 51.55%
*********************************
*********************************
   ACCURACIES FOR MODEL 2   
Overall Train	| Overall Test
99.95%		| 59.30%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 55.10%
1: 100.00%	| 1: 76.86%
2: 99.90%	| 2: 51.38%
3: 99.90%	| 3: 69.52%
4: 100.00%	| 4: 57.45%
5: 100.00%	| 5: 45.33%
6: 99.90%	| 6: 71.91%
7: 100.00%	| 7: 55.93%
8: 99.79%	| 8: 46.81%
9: 100.00%	| 9: 56.70%
*********************************
*********************************
   ACCURACIES FOR MODEL 3   
Overall Train	| Overall Test
99.97%		| 58.90%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 59.62%
1: 100.00%	| 1: 78.76%
2: 99.90%	| 2: 58.33%
3: 100.00%	| 3: 56.32%
4: 100.00%	| 4: 56.52%
5: 100.00%	| 5: 52.04%
6: 100.00%	| 6: 67.47%
7: 100.00%	| 7: 58.12%
8: 99.90%	| 8: 52.04%
9: 99.90%	| 9: 48.00%
*********************************
*********************************
   ACCURACIES FOR MODEL 4   
Overall Train	| Overall Test
99.93%		| 61.80%
----------------------------
Train Classwise	| Test Classwise
0: 100.00%	| 0: 65.59%
1: 100.00%	| 1: 66.94%
2: 99.90%	| 2: 62.83%
3: 99.91%	| 3: 69.72%
4: 99.80%	| 4: 67.74%
5: 100.00%	| 5: 56.04%
6: 100.00%	| 6: 61.62%
7: 100.00%	| 7: 61.05%
8: 99.68%	| 8: 53.66%
9: 100.00%	| 9: 49.50%
*********************************